In [18]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from mord import LogisticAT
from sklearn.linear_model import LinearRegression

In [10]:
pd.set_option('display.max_columns', None)
unfiltered_data=pd.read_csv('../raw_data/cleaned_data_040321.csv',encoding='latin-1')

In [11]:
X=unfiltered_data[['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11',
           'BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat',
          'Dem_maritalstatus','Dem_riskgroup','Dem_isolation'
         ]]
y=unfiltered_data["PSS10_avg"]

In [13]:
# X.head()

In [14]:
def edu_func(X):
    X['Dem_edu']=X['Dem_edu'].replace({'Uninformative response':0,'None':1,'Up to 6 years of school':2, 'Up to 9 years of school':3, 'Up to 12 years of school':4, 'Some College, short continuing education or equivalent':5, 'College degree, bachelor, master': 6, 'PhD/Doctorate':7 })
    return  X[['Dem_edu']]
def edu_mom_func(X):
    X['Dem_edu_mom'] = X['Dem_edu_mom'].replace({'Uninformative response':0,'None':1,'Up to 6 years of school':2, 'Up to 9 years of school':3, 'Up to 12 years of school':4, 'Some College or equivalent':5, 'College degree': 6, 'PhD/Doctorate':7 })
    return  X[['Dem_edu_mom']]
def dem_risk_group(X):
    X['Dem_riskgroup'] = X['Dem_riskgroup'].replace({'No':1,'Not sure':2, 'Yes':3})
    return  X[['Dem_riskgroup']]
def dem_expat_func(X):
    X['Dem_Expat'] = X['Dem_Expat'].replace({'no':0,'yes':1})
    return X[['Dem_Expat']]
#ohe = Pipeline([
 #   ('Onehotencoding', OneHotEncoder()) ])
    
imputing_values = ColumnTransformer([
    ('Dem_edu', FunctionTransformer(edu_func),['Dem_edu']),
    ('Dem_edu_mom',FunctionTransformer(edu_mom_func),['Dem_edu_mom']),
    ('Dem_risk_group',FunctionTransformer(dem_risk_group),['Dem_riskgroup']),
    ('Dem_Expat',FunctionTransformer(dem_expat_func),['Dem_Expat']),
    ('ohe_transformer', OneHotEncoder(), ['Dem_gender', 'Dem_maritalstatus','Dem_employment','Dem_isolation'])
],remainder='passthrough')

prepro_pipe = Pipeline([
    ('imputer', imputing_values),
    ('scaler', MinMaxScaler())])
#ohe_transformer=ColumnTransformer([
 #   ('ohe_transformer', ohe, ['Dem_gender', 'Dem_maritalstatus','ohe_isolation','Dem_employment'])
  #                           ])

# pd.DataFrame(num_transformer.fit_transform(X)).head()

In [15]:
from sklearn import set_config; set_config(display='diagram')
prepro_pipe

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Dem_edu',
                                                  FunctionTransformer(func=<function edu_func at 0x7f0c2db2f310>),
                                                  ['Dem_edu']),
                                                 ('Dem_edu_mom',
                                                  FunctionTransformer(func=<function edu_mom_func at 0x7f0c2db2f430>),
                                                  ['Dem_edu_mom']),
                                                 ('Dem_risk_group',
                                                  FunctionTransformer(func=<function dem_risk_group at 0x7f0c2db2f3a0>),
                                                  ['Dem_riskgroup']),
                                                 ('Dem_Expat',
                                                  FunctionTransformer(func=<function dem_expat_func at 0x7f0c2db2f5e0>),
                                                  ['Dem_Expat']),
                                                 ('ohe_transformer',
                                                  OneHotEncoder(),
                                                  ['Dem_gender',
                                                   'Dem_maritalstatus',
                                                   'Dem_employment',
                                                   'Dem_isolation'])])),
                ('scaler', MinMaxScaler())])

In [16]:
pd.DataFrame(prepro_pipe.fit_transform(X))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,0.857143,0.142857,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.2,0.2,0.8,0.8,0.8,0.2,0.8,1.0,0.8,0.2,0.6,1.0,0.8,0.2,0.8,0.021739
1,0.857143,0.857143,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.6,0.4,0.6,0.6,0.6,0.6,0.6,0.8,0.6,0.4,0.6,0.6,0.6,0.6,0.6,0.543478
2,0.714286,0.714286,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.8,0.6,0.8,0.8,0.6,0.8,0.8,0.8,0.8,0.4,0.8,0.8,0.8,0.4,0.8,0.119565
3,0.571429,0.571429,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.8,0.6,0.8,0.6,0.6,0.8,0.6,0.6,0.8,0.2,0.2,0.8,0.8,0.6,0.8,0.217391
4,0.857143,0.857143,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.8,0.2,0.8,0.6,0.6,0.6,0.8,0.8,0.8,0.2,0.4,1.0,1.0,0.0,1.0,0.141304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85484,0.857143,0.428571,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.2,0.8,0.4,0.4,0.4,1.0,0.8,0.6,0.2,1.0,1.0,0.8,0.0,1.0,0.434783
85485,0.857143,0.571429,0.5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.4,0.2,0.8,0.8,0.8,0.6,0.6,0.6,0.8,0.4,0.4,0.8,1.0,0.2,0.8,0.108696
85486,0.714286,0.571429,0.5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.8,0.8,0.4,0.2,0.6,0.8,0.6,0.8,1.0,0.6,0.6,0.8,0.6,0.6,0.4,0.163043
85487,0.571429,0.142857,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.2,0.4,0.8,0.6,0.6,0.2,0.0,0.8,0.8,1.0,0.2,1.0,0.423913


In [19]:
final_pipe = Pipeline([
    ('preproccessing', prepro_pipe),
    ('model', LinearRegression())
])



final_pipe

Pipeline(steps=[('preproccessing',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('Dem_edu',
                                                                   FunctionTransformer(func=<function edu_func at 0x7f0c2db2f310>),
                                                                   ['Dem_edu']),
                                                                  ('Dem_edu_mom',
                                                                   FunctionTransformer(func=<function edu_mom_func at 0x7f0c2db2f430>),
                                                                   ['Dem_edu_mom']),
                                                                  ('Dem_risk_group',
                                                                   FunctionTransformer(func=<function dem_risk_group at 0x7f0c2db2f3a0>),
                                                                   ['Dem_riskgroup']),
                                                                  ('Dem_Expat',
                                                                   FunctionTransformer(func=<function dem_expat_func at 0x7f0c2db2f5e0>),
                                                                   ['Dem_Expat']),
                                                                  ('ohe_transformer',
                                                                   OneHotEncoder(),
                                                                   ['Dem_gender',
                                                                    'Dem_maritalstatus',
                                                                    'Dem_employment',
                                                                    'Dem_isolation'])])),
                                 ('scaler', MinMaxScaler())])),
                ('model', LinearRegression())])

In [20]:
def get_user_data():
    values=[]

    columns=['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11',
           'BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat',
          'Dem_maritalstatus','Dem_riskgroup','Dem_isolation']

    for i in columns:
        value=random.randint(1,6)
        values.append(value)
    
    user_data=pd.DataFrame([values],columns=columns)
    return user_data

def get_data():
    return pd.read_csv('../raw_data/cleaned_data_040321.csv')

# def Xandy(df):
#     X=df[['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11','BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat','Dem_maritalstatus','Dem_riskgroup','Dem_isolation']]
#     y=df["PSS10_avg"]
#     return X,y

In [21]:
def model(X,y):
    model=LinearRegression()
    return knn.fit(X,y)
    

def predict(model):
    
    y_pred=model.predict(user_data)
    return y_pred

In [22]:
y

0        2.2
1        2.1
2        2.3
3        2.6
4        2.7
        ... 
85484    1.7
85485    1.7
85486    3.0
85487    1.4
85488    1.9
Name: PSS10_avg, Length: 85489, dtype: float64

In [23]:
# X=df[['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11','BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat','Dem_maritalstatus','Dem_riskgroup','Dem_isolation']]
# y=df["PSS10_avg"]

final_pipe.fit(X,y)


Pipeline(steps=[('preproccessing',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('Dem_edu',
                                                                   FunctionTransformer(func=<function edu_func at 0x7f0c2db2f310>),
                                                                   ['Dem_edu']),
                                                                  ('Dem_edu_mom',
                                                                   FunctionTransformer(func=<function edu_mom_func at 0x7f0c2db2f430>),
                                                                   ['Dem_edu_mom']),
                                                                  ('Dem_risk_group',
                                                                   FunctionTransformer(func=<function dem_risk_group at 0x7f0c2db2f3a0>),
                                                                   ['Dem_riskgroup']),
                                                                  ('Dem_Expat',
                                                                   FunctionTransformer(func=<function dem_expat_func at 0x7f0c2db2f5e0>),
                                                                   ['Dem_Expat']),
                                                                  ('ohe_transformer',
                                                                   OneHotEncoder(),
                                                                   ['Dem_gender',
                                                                    'Dem_maritalstatus',
                                                                    'Dem_employment',
                                                                    'Dem_isolation'])])),
                                 ('scaler', MinMaxScaler())])),
                ('model', LinearRegression())])

In [24]:
pd.DataFrame(X.iloc[-1]).T

,BFF_15_1,BFF_15_2,BFF_15_3,BFF_15_4,BFF_15_5,BFF_15_6,BFF_15_7,BFF_15_8,BFF_15_9,BFF_15_10,BFF_15_11,BFF_15_12,BFF_15_13,BFF_15_14,BFF_15_15,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,Dem_Expat,Dem_maritalstatus,Dem_riskgroup,Dem_isolation
85488,4.0,4.0,3.0,3.0,3.0,5.0,3.0,5.0,5.0,3.0,2.0,3.0,5.0,4.0,4.0,54,Male,"Some College, short continuing education or eq...",Some College or equivalent,Full time employed,yes,Married/cohabiting,Not sure,Life carries on with minor changes
